In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
#Store Part 1 results into data frame
cities_results_df=pd.read_csv("../output_data/cities.csv")
cities_results_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,lompoc,34.6391,-120.4579,53.60,82,1,16.11,US,1613182327
1,atuona,-9.8000,-139.0333,78.93,79,1,15.77,PF,1613182328
2,mataura,-46.1927,168.8643,73.99,52,85,1.99,NZ,1613182329
3,hermanus,-34.4187,19.2345,72.00,79,0,3.00,ZA,1613182330
4,palmer,42.1584,-72.3287,23.00,45,90,2.10,US,1613182148


In [4]:
#Humidity Heat maps
#Configure gmaps
gmaps.configure(api_key=g_key)
#get lat and lng and humidity
locations = cities_results_df[["Lat", "Lng"]]
rating = cities_results_df["Humidity"].astype(float)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=max(rating),
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]